# 01_experiments_dev — MVTec 画像異常検知（開発

本ノートは dev カテゴリのみで設計を確定し、固定パイプライン設定を `assets/fixed_pipeline.json` に出力するためのテンプレートです。
- データ取得は anomalib を用いる（AGENTS.md 準拠）
- 手法は Mahalanobis / PaDiM を比較
- 閾値は dev の test で画像レベル FPR=1% を目標に決定

実行順序：Header → Data → Methods → Results → Save JSON/Artifacts


## 環境・依存の読み込み

In [ ]:
# 参考: anomaly_detection.ipynb からの初期インポートを整理
import os
from pathlib import Path
import json
import numpy as np
import pandas as pd
import torch
from torchvision import models, transforms
from sklearn.covariance import ledoit_wolf
import matplotlib.pyplot as plt
# 可視化などのユーティリティは必要に応じて追加
from torch import nn
import torch.nn.functional as F


## データ取得（anomalib 経由）

In [ ]:
# AGENTS.md: 既存の MVTEC_ROOT または data/mvtec を使用。
# 未検出の場合は anomalib によりダウンロード。
MVTEC_ROOT = Path(os.environ.get("MVTEC_ROOT", "data/mvtec"))
MVTEC_ROOT.mkdir(parents=True, exist_ok=True)

# anomalib のAPIはバージョンで異なる可能性があるため、例示的に記述。
# 実環境の anomalib バージョンに合わせて import と引数を調整してください。
try:
    from anomalib.data import MVTecAD
    datamodule = MVTecAD(root=str(MVTEC_ROOT))
    datamodule.prepare_data()  # download if needed
    datamodule.setup()
except Exception as e:
    print("[WARN] anomalib のデータ取得セットアップで問題が発生しました。バージョンや引数を確認してください:\n", e)

assert MVTEC_ROOT.exists(), "MVTec root not found after anomalib setup."


## 実験設定（dev のみ）

In [ ]:
# dev カテゴリと seed を定義
dev_category = "carpet"  # 例: AGENTS.md 推奨例
seeds = [0, 1, 2]
image_size = 256

# 比較する手法（最小構成）
use_mahalanobis = True
use_padim = True

# PaDiM や Mahalanobis で用いる backbone/layers 等は仮パラメータ（要調整）
backbone = "resnet18"
padim_layers = ["layer2", "layer3"]
padim_channel_subsample = 100
cov_estimator = "ledoit_wolf"  # Mahalanobis 用


## Methods — Mahalanobis / PaDiM（テンプレート）
- ここで特徴抽出（ImageNet 事前学習）/ 統計量推定 / 推論スコア化を実装します。
- 本テンプレートでは骨子のみを用意しています。必要に応じて `anomaly_detection.ipynb` の実装を移植してください。

In [ ]:
def fit_mahalanobis(train_loader, backbone=backbone, cov_estimator=cov_estimator):
    """Fit Mahalanobis model from training data."""
    model = models.__dict__[backbone](pretrained=True)
    feature_extractor = nn.Sequential(*list(model.children())[:-1])
    feature_extractor.eval()
    feats = []
    device = next(feature_extractor.parameters()).device
    with torch.no_grad():
        for images, _ in train_loader:
            images = images.to(device)
            feat = feature_extractor(images).view(images.size(0), -1)
            feats.append(feat.cpu().numpy())
    feats = np.concatenate(feats, axis=0)
    mean = feats.mean(axis=0)
    cov, _ = ledoit_wolf(feats)
    precision = np.linalg.pinv(cov)
    return {
        "mean": mean,
        "precision": precision,
        "feature_extractor": feature_extractor,
        "meta": {"backbone": backbone, "cov_estimator": cov_estimator},
    }


def score_mahalanobis(model_state, batch):
    """Return Mahalanobis distances for a batch."""
    feature_extractor = model_state["feature_extractor"]
    device = next(feature_extractor.parameters()).device
    mean = torch.tensor(model_state["mean"], device=device)
    precision = torch.tensor(model_state["precision"], device=device)
    feature_extractor.eval()
    images = batch[0] if isinstance(batch, (tuple, list)) else batch
    images = images.to(device)
    with torch.no_grad():
        feats = feature_extractor(images).view(images.size(0), -1)
    diff = feats - mean
    scores = torch.sqrt(torch.sum((diff @ precision) * diff, dim=1))
    return scores.cpu()


class _PadimFeatureExtractor(nn.Module):
    """Utility to capture intermediate feature maps."""

    def __init__(self, backbone, layers):
        super().__init__()
        self.model = models.__dict__[backbone](pretrained=True)
        self.layers = layers
        self.outputs = {}
        for name, module in self.model.named_children():
            if name in layers:
                module.register_forward_hook(self._save_output(name))

    def _save_output(self, name):
        def hook(module, inp, out):
            self.outputs[name] = out
        return hook

    def forward(self, x):
        self.outputs = {}
        _ = self.model(x)
        return [self.outputs[l] for l in self.layers]


def fit_padim(train_loader, backbone=backbone, layers=padim_layers, d=padim_channel_subsample):
    """Fit PaDiM model and return per-location statistics."""
    feature_extractor = _PadimFeatureExtractor(backbone, layers)
    feature_extractor.eval()
    embedding_list = []
    device = next(feature_extractor.parameters()).device
    with torch.no_grad():
        for images, _ in train_loader:
            images = images.to(device)
            feats = feature_extractor(images)
            feats = [
                F.interpolate(f, size=feats[0].shape[-2:], mode="bilinear", align_corners=False)
                for f in feats
            ]
            embedding = torch.cat(feats, dim=1)
            embedding_list.append(embedding.cpu())
    embeddings = torch.cat(embedding_list, dim=0)
    c = embeddings.shape[1]
    h, w = embeddings.shape[2:]
    torch.manual_seed(0)
    idx = torch.randperm(c)[:d]
    embeddings = embeddings[:, idx, :, :]
    embeddings = embeddings.permute(0, 2, 3, 1).reshape(-1, h * w, d)
    mean = embeddings.mean(dim=0)
    cov = torch.zeros(h * w, d, d)
    for i in range(h * w):
        cov[i] = torch.from_numpy(np.cov(embeddings[:, i, :].T))
    return {
        "mean": mean,
        "cov": cov,
        "idx": idx,
        "feature_extractor": feature_extractor,
        "meta": {"backbone": backbone, "layers": layers, "d": d},
    }


def score_padim(model_state, batch):
    """Return PaDiM anomaly scores for batch."""
    feature_extractor = model_state["feature_extractor"]
    device = next(feature_extractor.parameters()).device
    mean = model_state["mean"].to(device)
    cov = model_state["cov"].to(device)
    idx = model_state["idx"]
    feature_extractor.eval()
    images = batch[0] if isinstance(batch, (tuple, list)) else batch
    images = images.to(device)
    with torch.no_grad():
        feats = feature_extractor(images)
        feats = [
            F.interpolate(f, size=feats[0].shape[-2:], mode="bilinear", align_corners=False)
            for f in feats
        ]
        embedding = torch.cat(feats, dim=1)[:, idx, :, :]
    n, d, h, w = embedding.shape
    embedding = embedding.permute(0, 2, 3, 1).reshape(n, h * w, d)
    scores = []
    for emb in embedding:
        dist = []
        for i in range(h * w):
            diff = emb[i] - mean[i]
            inv = torch.linalg.pinv(cov[i])
            dist.append(torch.sqrt(diff @ inv @ diff))
        dist = torch.stack(dist).reshape(h, w)
        scores.append(dist.max())
    return torch.stack(scores).cpu()


## Results — Cross-Validation & Metrics（テンプレート）
- 各手法で dev の train を用いたクロスバリデーションを実施し、訓練内/訓練外スコアのヒストグラムを確認する。
- dev の test で閾値と評価指標（AUROC, F1 など）の関係を可視化する。


In [ ]:
# TODO: implement cross-validation loops and metric computation
# for seed in seeds:
#     # split dev train into folds
#     # fit models and compute scores
#     # plot histograms and metric curves
#     pass
pass


### 閾値決定（FPR=1% 目標、dev の test のみ）

In [ ]:
# NOTE: ここでは閾値決定の骨子のみを用意。
# 実装時は dev の test スコア分布から FPR=1% となるスコアを求めてください。
image_fpr_target = 0.01
threshold_value = None  # TODO: compute from dev test scores
threshold_source = f"dev_{dev_category}_test"
print("[INFO] threshold_source:", threshold_source)


## 固定パイプラインの保存（assets/fixed_pipeline.json）

In [ ]:
# デフォルトではファイルを書き出さない（テンプレートのため）。
# 実際に保存したい場合は SAVE_FIXED=True にして実行してください。
SAVE_FIXED = False

fixed_pipeline = {
    "common": {"image_size": image_size, "seeds": seeds},
    "threshold": {"image_fpr_target": image_fpr_target, "value": threshold_value, "source": threshold_source},
    "mahalanobis": {"backbone": backbone, "cov_estimator": cov_estimator},
    "padim": {"layers": padim_layers, "channel_subsample": padim_channel_subsample}
}

assets_dir = Path("assets")
assets_dir.mkdir(parents=True, exist_ok=True)
cfg_path = assets_dir / "fixed_pipeline.json"

if SAVE_FIXED:
    with cfg_path.open("w", encoding="utf-8") as f:
        json.dump(fixed_pipeline, f, indent=2, ensure_ascii=False)
    print(f"[INFO] Saved: {cfg_path}")
else:
    print("[INFO] SAVE_FIXED=False のためファイルは出力しません。")


## 次の手順
- 上記のテンプレート関数に実装を追加し、dev の test から閾値を決めて `SAVE_FIXED=True` で JSON を保存。
- その後 `02_evaluation_report.ipynb` で eval カテゴリを一発評価。
- リーク防止のため、02 ではパラメータ・閾値を変更しないこと。
